# Entry 2: Continued Fraction Convergence Analysis of En

This notebook analyzes the continued fraction (CF) expansions of En, the golden ratio φ, and Euler’s number e. It compares:

- **Structural patterns**
- **Convergence rates**
- **Symbolic variance**
- **Divergence sequences**

The analysis is performed at 400-digit precision.

---

### Summary of Methods:
- En is defined as:

\[
E_n = \sqrt{\frac{\ln(\pi)}{\ln(e / (e - 1))}}
\]

- Continued fractions are extracted via custom algorithms
- Convergents are plotted over 500 terms
- Variance of CF terms is computed to measure symbolic structure
- Difference sequences show drift behavior

This work aims to position En within the symbolic space of known transcendental constants.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal, getcontext
from mpmath import mp

# Set high precision
getcontext().prec = 400
mp.dps = 400  # mpmath for better π

# Compute e
def compute_e(terms=100):
    one = Decimal('1')
    e_approx = one
    factorial = one
    for n in range(1, terms + 1):
        factorial *= n
        e_approx += one / factorial
    return e_approx

# Use mpmath for π (Chudnovsky algorithm internally)
pi = Decimal(str(mp.pi()))
e = compute_e()
one = Decimal('1')
phi = (one + Decimal('5').sqrt()) / Decimal('2')
En = Decimal.sqrt(Decimal.ln(pi) / Decimal.ln(e / (e - one)))

# Continued fraction with improved stability
def get_continued_fraction(x, max_terms=1000):
    cf = []
    a = x
    for i in range(max_terms):
        integer_part = int(a)
        cf.append(integer_part)
        fractional_part = a - integer_part
        if fractional_part < Decimal('1e-390'):
            break
        a = Decimal('1') / fractional_part
        if a > Decimal('1e390'):
            break
    while len(cf) < max_terms:
        cf.append(1)
    return cf

# Convergents
def compute_convergents(cf):
    h, k = [0, 1], [1, 0]
    convergents = []
    for a in cf:
        h.append(a * h[-1] + h[-2])
        k.append(a * k[-1] + k[-2])
        convergents.append(Decimal(h[-1]) / Decimal(k[-1]))
    return convergents

# Fixed variance
def custom_variance(int_list):
    n = len(int_list)
    mean = sum(int_list) / n
    squared_diff_sum = sum((x - mean) ** 2 for x in int_list)
    return int(squared_diff_sum / n) if squared_diff_sum > 0 else 0

# CF differences
def compute_cf_differences(cf):
    return [cf[i + 1] - cf[i] for i in range(len(cf) - 1)]

# Compute CFs
cf_En = get_continued_fraction(En)
cf_phi = get_continued_fraction(phi)
cf_e = get_continued_fraction(e)
En_differences = compute_cf_differences(cf_En)

# 1000 terms
n_terms = 1000
n_range = np.arange(1, n_terms + 1)
conv_En = compute_convergents(cf_En)[:n_terms]
conv_phi = compute_convergents(cf_phi)[:n_terms]
conv_e = compute_convergents(cf_e)[:n_terms]
cf_En = cf_En[:n_terms]
cf_phi = cf_phi[:n_terms]
cf_e = cf_e[:n_terms]

# Convert for plotting
conv_En_float = [float(x) for x in conv_En]
conv_phi_float = [float(x) for x in conv_phi]
conv_e_float = [float(x) for x in conv_e]
En_float = float(En)
phi_float = float(phi)
e_float = float(e)

# Plot (500 terms)
plot_terms = 500
plt.figure(figsize=(14, 8))
plt.plot(n_range[:plot_terms], conv_En_float[:plot_terms], label=f'En → {En_float:.6f}', marker='o', color='b')
plt.plot(n_range[:plot_terms], conv_phi_float[:plot_terms], label=f'φ → {phi_float:.6f}', marker='x', color='g')
plt.plot(n_range[:plot_terms], conv_e_float[:plot_terms], label=f'e → {e_float:.6f}', marker='s', color='r')
plt.axhline(y=En_float, color='b', linestyle='--')
plt.axhline(y=phi_float, color='g', linestyle='--')
plt.axhline(y=e_float, color='r', linestyle='--')
plt.xlabel('Term (n)')
plt.ylabel('Value')
plt.title('Convergence (First 500 Terms)')
plt.legend()
plt.grid(True)
plt.show()

# Variances
var_En = custom_variance(cf_En)
var_phi = custom_variance(cf_phi)
var_e = custom_variance(cf_e)

# Results
print(f"Variance En: {var_En}")
print(f"Variance φ: {var_phi}")
print(f"Variance e: {var_e}")
print(f"En CF (first 10): {cf_En[:10]}")
print(f"En Differences (first 10): {En_differences[:10]}")
